In [6]:
from datetime import datetime
import itertools
import json
import math
import os
import pickle as pkl

import geopandas
from haversine import haversine_vector, Unit
import numpy as np
import pandas as pd

from pandarallel import pandarallel
pandarallel.initialize()

INFO: Pandarallel will run on 4 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [7]:
school_year = "2017_2018"
schools_df = geopandas.read_file("LCPS_data/LCPS_Sites_%s.shp" % school_year)
spas_df = geopandas.read_file("LCPS_data/PlanningZones_%s.shp" % school_year)
students_df = geopandas.read_file("LCPS_data/Students_%s.shp" % school_year)

schools_df = schools_df.to_crs(epsg=4326)  # This gives us proper lat and long in the WGS84 ellipsoid.
spas_df = spas_df.to_crs(schools_df.crs)
students_df = students_df.to_crs(schools_df.crs)

In [8]:
schools_df = schools_df[(schools_df.CLASS == "ELEMENTARY") | (schools_df.CLASS == "MIDDLE") | (schools_df.CLASS == "HIGH")]
schools_df = schools_df[(schools_df.NAME != "DOUGLASS COMMUNITY") & (schools_df.NAME != "CS MONROE TECHNOLOGY")]

es_schools_df = schools_df[schools_df.CLASS == "ELEMENTARY"]
ms_schools_df = schools_df[schools_df.CLASS == "MIDDLE"]
hs_schools_df = schools_df[schools_df.CLASS == "HIGH"]

In [9]:
weights = list(range(0, 11))
X_S = ["ES", "MS", "HS"]
shc_results = dict(zip(X_S, [dict() for S in X_S]))

for weight, school_level in itertools.product(weights, X_S):
    with open("results/SHC/run%d_%s_SHC.json" % (weight, school_level), "r") as results_file:
        shc_results[school_level][weight] = json.load(results_file)

In [10]:
# These 3 lines take ~1 minute on 8 hyperthreads.
students_df = students_df[students_df.GRADE != 14]
loudoun_polygon = spas_df.unary_union
students_df = students_df[students_df.geometry.parallel_map(loudoun_polygon.contains)]

es_students_df = students_df[(students_df.GRADE <= 5) | (students_df.GRADE == 13)]
ms_students_df = students_df[(students_df.GRADE >= 6) & (students_df.GRADE <= 8)]
hs_students_df = students_df[(students_df.GRADE >= 9) & (students_df.GRADE <= 12)]

def group_students_by_spa(student):
    """GroupBy function for gathering student records to their SPAs.
    
    Args:
        student: integer row index of the student record
    
    Returns:
        A label to group this record into (the student's SPA).
    """
    spa_index = spas_df.geometry.map(lambda spa: spa.contains(students_df.loc[student].geometry))
    spa_index = spa_index[spa_index == True]
    if len(spa_index) == 0:
        return "NONE"
    assert(len(spa_index) == 1)
    return spas_df.loc[spa_index.index[0]].STDYAREA

groups_path = "data/spa_groups.json"
if os.path.exists(groups_path):
    with open(groups_path, "r") as fp:
        spa_groups = json.load(fp)
else:
    # This is O(k * n) overall for k SPAs and n students.
    # Expect ~2.25 hours on a laptop or 1.5 on a server.
    tic = datetime.now()
    spa_groups = students_df.groupby(by=group_students_by_spa).groups
    toc = datetime.now()
    print(toc - tic)

    def serialize_groups(obj):
        if isinstance(obj, pd.Int64Index):
            return list(obj)
        else:
            raise Exception("obj was not an Int64Index as expected...")

    with open(groups_path, "w") as fp:
        json.dump(spa_groups, fp, default=serialize_groups)

In [11]:
# This has the form of Dict[school_level: str][weight: int] = List[trial: float]
# Utilization scores: lower is better
utilization_scores = dict(zip(X_S, [dict() for S in X_S]))

sigma = 5
for school_level in X_S:
    for weight in weights:
        final_partitions = [shc_results[school_level][weight][str(trial)]["info"]["Final"]["zones"] for trial in range(1, 26)]
        trial_utilizations = []

        for i, trial in enumerate(final_partitions, start=1):
            utilization = 0
            for school, school_district in trial.items():
                school_population = school_district["Population"]
                school_capacity = school_district["Capacity"]
                utilization += abs(math.atan(sigma * (1 - school_population / school_capacity)))
            utilization /= ((math.pi / 2) * len(trial.keys()))
            assert(utilization >= 0 and utilization <= 1)
            trial_utilizations.append(utilization) 
        utilization_scores[school_level][weight] = trial_utilizations

In [12]:
# Socioeconomic scores: lower is better
socioeconomic_scores = dict(zip(X_S, [dict() for S in X_S]))

# FSI value of 1 or 2 indicates free/reduced meals.
county_farm_rate = dict()
county_farm_rate["ES"] = len(es_students_df[es_students_df.FSI <= 2]) / len(es_students_df)
county_farm_rate["MS"] = len(ms_students_df[ms_students_df.FSI <= 2]) / len(ms_students_df)
county_farm_rate["HS"] = len(hs_students_df[hs_students_df.FSI <= 2]) / len(hs_students_df)

def count_school_farms(spa_name, school_level):
    try:
        spa_students = students_df.loc[spa_groups[spa_name]]
    except:
        spa = spas_df[spas_df.STDYAREA == spa_name].iloc[0]
        assert spa.TOTAL_KG_5 + spa.TOTAL_6_8 + spa.TOTAL_9_12 == 0
        return 0
    if school_level == "ES":
        spa_students = spa_students[(spa_students.GRADE <= 5) | (spa_students.GRADE == 13)]
    elif school_level == "MS":
        spa_students = spa_students[(spa_students.GRADE >= 6) & (spa_students.GRADE <= 8)]
    elif school_level == "HS":
        spa_students = spa_students[(spa_students.GRADE >= 9) & (spa_students.GRADE <= 12)]
    return len(spa_students[spa_students.FSI <= 2])

normalization = dict()
for level in X_S:
    normalization[level] = max([1 - county_farm_rate[level], county_farm_rate[level]])

# Expect ~38 minutes.
tic = datetime.now()
for school_level in X_S:
    for weight in weights:
        final_partitions = [shc_results[school_level][weight][str(trial)]["info"]["Final"]["zones"] for trial in range(1, 26)]
        trial_economic = []

        for i, trial in enumerate(final_partitions, start=1):
            economic = 0
            for school, school_district in trial.items():
                school_farm_students = sum([count_school_farms(spa, school_level) for spa in school_district["STATE"]])
                if school_district["Population"] == 0:
                    # There are some strange plans where nobody was assigned to a school...
                    economic += county_farm_rate[school_level]
                else:
                    economic += abs(school_farm_students / school_district["Population"] - county_farm_rate[school_level]) / normalization[level]
            economic /= len(trial.keys())
            assert economic >= 0 and economic <= 1, economic
            trial_economic.append(economic)
        socioeconomic_scores[school_level][weight] = trial_economic
toc = datetime.now()
print(toc - tic)

0:27:21.144391


In [13]:
# Diversity scores: lower is better
diversity_scores = dict(zip(X_S, [dict() for S in X_S]))

county_entropy = dict()
county_entropy["ES"] = es_students_df.ETHNIC.value_counts() / len(es_students_df)
county_entropy["ES"] = county_entropy["ES"].apply(lambda value: value * np.log(1 / value)).sum()
county_entropy["MS"] = ms_students_df.ETHNIC.value_counts() / len(ms_students_df)
county_entropy["MS"] = county_entropy["MS"].apply(lambda value: value * np.log(1 / value)).sum()
county_entropy["HS"] = hs_students_df.ETHNIC.value_counts() / len(hs_students_df)
county_entropy["HS"] = county_entropy["HS"].apply(lambda value: value * np.log(1 / value)).sum()

county_pop = {"ES": len(es_students_df), "MS": len(ms_students_df), "HS": len(hs_students_df)}

def query_ethnicity_column(spa_name, school_level):
    try:
        spa_students = students_df.loc[spa_groups[spa_name]]
    except:
        spa = spas_df[spas_df.STDYAREA == spa_name].iloc[0]
        assert spa.TOTAL_KG_5 + spa.TOTAL_6_8 + spa.TOTAL_9_12 == 0
        return pd.Series(np.zeros(7), index=students_df.ETHNIC.unique(), name="ETHNIC")
    if school_level == "ES":
        spa_students = spa_students[(spa_students.GRADE <= 5) | (spa_students.GRADE == 13)]
    elif school_level == "MS":
        spa_students = spa_students[(spa_students.GRADE >= 6) & (spa_students.GRADE <= 8)]
    elif school_level == "HS":
        spa_students = spa_students[(spa_students.GRADE >= 9) & (spa_students.GRADE <= 12)]
    return spa_students.ETHNIC.value_counts()

# Expect ~45 minutes.
tic = datetime.now()
for school_level in X_S:
    for weight in weights:
        final_partitions = [shc_results[school_level][weight][str(trial)]["info"]["Final"]["zones"] for trial in range(1, 26)]
        trial_diversity = []

        for i, trial in enumerate(final_partitions, start=1):
            diversity = 0
            for school, school_district in trial.items():
                spa_list = pd.DataFrame()
                for spa in school_district["STATE"]:
                    counts = query_ethnicity_column(spa, school_level)
                    spa_list = pd.concat([spa_list, counts.to_frame().T])

                entropy = spa_list.sum(axis=0)
                if school_district["Population"] == 0:
                    continue
                entropy = entropy / school_district["Population"]
                def calc_entropy(value):
                    if value == 0 or np.isnan(value):
                        return 0
                    return value * np.log(1 / value)
                entropy = entropy.apply(calc_entropy).sum()

                diversity += (school_district["Population"] * (county_entropy[school_level] - entropy)) / (county_entropy[school_level] * county_pop[school_level])

            assert diversity >= 0 and diversity <= 1, "%s, %s" % (str(diversity), school_district["Population"])
            trial_diversity.append(diversity)
        diversity_scores[school_level][weight] = trial_diversity
toc = datetime.now()
print(toc - tic)

/Users/jeffchan/.vre/36/lib/python3.6/site-packages/ipykernel_launcher.py:42: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False



0:43:51.289229


In [14]:
# To compute walkability, we need to compute a distance matrix:
# This lookup table will tell us the distances of every student to every school.
# This will in turn be used to compute the number of students within walking distance of their nearest school.

# Due to numerical stability, the Haversine Formula is needed for GPS distances <= 1 km.
# We don't need to worry about antipodal instabilities in the traditional formulation.
# The haversine package most likely makes assumptions about the ellipsoid CRS.
# (I.e. they probably use a static radius of the earth, losing precision with the GIS CRS.)
# This should be sufficiently accurate for now.  
# # TODO: improve precision if necessary

# Get this data in miles.
es_students_coords = list(es_students_df.geometry.map(lambda point: point.coords[0]))
es_schools_coords = list(es_schools_df.geometry.map(lambda point: point.coords[0]))
es_distance = pd.DataFrame(haversine_vector(es_students_coords, es_schools_coords, Unit.MILES, comb=True).transpose(), index=es_students_df.index, columns=es_schools_df.SCH_CODE)

ms_students_coords = list(ms_students_df.geometry.map(lambda point: point.coords[0]))
ms_schools_coords = list(ms_schools_df.geometry.map(lambda point: point.coords[0]))
ms_distance = pd.DataFrame(haversine_vector(ms_students_coords, ms_schools_coords, Unit.MILES, comb=True).transpose(), index=ms_students_df.index, columns=ms_schools_df.SCH_CODE)

hs_students_coords = list(hs_students_df.geometry.map(lambda point: point.coords[0]))
hs_schools_coords = list(hs_schools_df.geometry.map(lambda point: point.coords[0]))
hs_distance = pd.DataFrame(haversine_vector(hs_students_coords, hs_schools_coords, Unit.MILES, comb=True).transpose(), index=hs_students_df.index, columns=hs_schools_df.SCH_CODE)

walkable_distance = {"ES": 1, "MS": 1.25, "HS": 1.25}
es_walkable = (es_distance.min(axis=1) <= walkable_distance["ES"]).value_counts()[True]
ms_walkable = (ms_distance.min(axis=1) <= walkable_distance["MS"]).value_counts()[True]
hs_walkable = (hs_distance.min(axis=1) <= walkable_distance["HS"]).value_counts()[True]
county_walkable = {"ES": es_walkable, "MS": ms_walkable, "HS": hs_walkable}

In [15]:
walkability_scores = dict(zip(X_S, [dict() for S in X_S]))
students = {"ES": es_students_df, "MS": ms_students_df, "HS": hs_students_df}
distances = {"ES": es_distance, "MS": ms_distance, "HS": hs_distance}

tic = datetime.now()
for school_level in X_S:
    for weight in weights:
        final_partitions = [shc_results[school_level][weight][str(trial)]["info"]["Final"]["zones"] for trial in range(1, 26)]
        trial_walkability = []

        for i, trial in enumerate(final_partitions, start=1):
            students_in_walking_distance = 0
            for school, school_district in trial.items():
                student_indices = np.concatenate([(np.array(spa_groups[spa_name], dtype=np.int32) if spa_name in spa_groups else np.array([])) for spa_name in school_district["STATE"]])
                assert len(student_indices.shape) == 1
                student_indices = students[school_level].index.isin(student_indices)
                
                distance_from_school = (distances[school_level].loc[student_indices])[school]

                in_distance = (distance_from_school <= walkable_distance[school_level])
                if in_distance.any():
                    students_in_walking_distance += in_distance.value_counts()[True]
                
            walkability = 1 - students_in_walking_distance / county_walkable[school_level]
            assert(walkability >= 0 and walkability <= 1)
            trial_walkability.append(walkability) 
        walkability_scores[school_level][weight] = trial_walkability
toc = datetime.now()
print(toc - tic)

0:01:02.330060


In [16]:
# We would like baseline statistics to compare these results to.
# How well do these heuristics do against the existing school zones?
schools = {"ES": es_schools_df, "MS": ms_schools_df, "HS": hs_schools_df}

existing_plan = dict()
for level in X_S:
    existing_plan[level] = shc_results[level][7]["1"]["info"]["Final"]["zones"]

utilization_baseline = dict()
for level in X_S:
    utilization = 0
    for school, zone in existing_plan[level].items():
        school_population = zone["Population"]
        school_capacity = zone["Capacity"]
        utilization += abs(math.atan(sigma * (1 - school_population / school_capacity)))
    utilization /= ((math.pi / 2) * len(schools[level]))
    assert utilization >= 0 and utilization <= 1
    utilization_baseline[level] = utilization

socioeconomic_baseline = dict()
for level in X_S:
    economic = 0
    for school, zone in existing_plan[level].items():
        school_farm_students = sum([count_school_farms(spa, school_level) for spa in zone["STATE"]])
        if zone["Population"] == 0:
            # There are some strange plans where nobody was assigned to a school...
            economic += county_farm_rate[level]
        else:
            economic += abs(school_farm_students / zone["Population"] - county_farm_rate[level]) / normalization[level]
    economic /= len(schools[level])
    assert economic >= 0 and economic <= 1, economic
    socioeconomic_baseline[level] = economic

diversity_baseline = dict()
for level in X_S:
    diversity = 0
    for school, zone in existing_plan[level].items():
        spa_list = pd.DataFrame()
        for spa in zone["STATE"]:
            counts = query_ethnicity_column(spa, level)
            spa_list = pd.concat([spa_list, counts.to_frame().T])

        entropy = spa_list.sum(axis=0)
        if zone["Population"] == 0:
            continue
        entropy = entropy / zone["Population"]
        def calc_entropy(value):
            if value == 0 or np.isnan(value):
                return 0
            return value * np.log(1 / value)
        entropy = entropy.apply(calc_entropy).sum()

        diversity += (zone["Population"] * (county_entropy[level] - entropy)) / (county_entropy[level] * county_pop[level])

    assert diversity >= 0 and diversity <= 1, "%s, %s" % (str(diversity), zone["Population"])
    diversity_baseline[level] = diversity

walkability_baseline = dict()
for level in X_S:
    students_in_walking_distance = 0
    for school, zone in existing_plan[level].items():
        student_indices = np.concatenate([(np.array(spa_groups[spa_name], dtype=np.int32) if spa_name in spa_groups else np.array([])) for spa_name in zone["STATE"]])
        assert len(student_indices.shape) == 1
        student_indices = students[level].index.isin(student_indices)
        
        distance_from_school = (distances[level].loc[student_indices])[school]

        in_distance = (distance_from_school <= walkable_distance[level])
        if in_distance.any():
            students_in_walking_distance += in_distance.value_counts()[True]
        
    walkability = 1 - students_in_walking_distance / county_walkable[level]
    assert walkability >= 0 and walkability <= 1
    walkability_baseline[level] = walkability

baseline_scores = {"utilization": utilization_baseline, "socioeconomic": socioeconomic_baseline, "diversity": diversity_baseline, "walkability": walkability_baseline}

/Users/jeffchan/.vre/36/lib/python3.6/site-packages/ipykernel_launcher.py:41: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False



In [20]:
DIR = 'metric_scores/'
if os.path.exists(DIR) == False:
    os.makedirs(DIR)
metrics = ['utilization_scores', 'socioeconomic_scores', 'diversity_scores', 'walkability_scores', 'baseline_scores']
for m in metrics:
    saved_PATH = "{}{}.pkl".format(DIR, m)
    with open(saved_PATH, 'wb') as f:
        print(saved_PATH)
        pkl.dump(locals()[m], f)

metric_scores/utilization_scores.pkl
metric_scores/socioeconomic_scores.pkl
metric_scores/diversity_scores.pkl
metric_scores/walkability_scores.pkl
metric_scores/baseline_scores.pkl
